### 19.3 implement synchronization for two interleaving threads

Thread ***t1*** prints odd numbers from 1 to 100;  Thread ***t2*** prints even numbers from 1 to 100.
Write code in which the two threads, running concurrently, print the numbers from 1 to 100 in order.

***Hint***:  The two threads need to notify each other when they are done.

In [2]:
import threading
import sys

class OddEvenMonitor(threading.Condition):
    
    ODD_TURN = True
    EVEN_TURN = False
    
    def __init__(self):
        super().__init__()
        self.turn = self.ODD_TURN
        
    def wait_turn(self, old_turn):
        with self:
            while self.turn != old_turn:
                self.wait()
                
    def toggle_turn(self):
        with self:
            self.turn ^= True
            self.notify()

class OddThread(threading.Thread):
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor

    def run(self):
        for i in range(1, 101, 2):
            self.monitor.wait_turn(OddEvenMonitor.ODD_TURN)
            sys.stdout.write("{} ".format(i))
            self.monitor.toggle_turn()            

class EvenThread(threading.Thread):
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor

    def run(self):
        for i in range(2, 101, 2):
            self.monitor.wait_turn(OddEvenMonitor.EVEN_TURN)
            sys.stdout.write("{} ".format(i))
            self.monitor.toggle_turn()            
           

monitor = OddEvenMonitor()
odd = OddThread(monitor)
even = EvenThread(monitor)

odd.start()
even.start()
odd.join()
even.join()


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

### Remarks:

Okay, it seems like the **wait_turn** keeps each thread waiting for the monitor's turn, which is either ***ODD_TURN*** or ***EVEN_TURN***.  This allows each thread, once the monitor says it can go to execute its own write to stdout.  It then updates the monitor by toggling the turn from ***True*** to ***False***.
Even though each **run** for each thread class is in a loop.  Each iteration of the loop waits for its turn, writes to stdout and then toggles the turn.